In [1]:
# Initialisation
if __name__=='__main__':
    %run '../ruaridhsNotebookPrettifier.ipy'
    %run '../ruaridhsNotebookInitialiser.ipy'

"improved" the css


toggled the visibility of header and toolbar
imported pandas as pd | imported numpy as np | imported ruaridhsNotebookImporter | imported matplotlib.pyplot as plt and changed visual style to 'ggplot' | set matplotlib plots to display inline at retina resolution | imported nitpick.plots as nplt | imported bokeh.charts and bokeh.plotting as bkc and bkp | 

Loading BokehJS ...

In [14]:
import sys
sys.path.insert(0, '../')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline
%config InlineBackend.figure_format='retina'
import bokeh.plotting as bkp

import ruaridhsNotebookImporter

# Define Deep Network Class
This creates a class for training deep networks (with 2D training data, and 2 classes). It trains netorks, logs information during training and finally generates the final plots of the decision boundary.

In [16]:
import framework as fk

In [17]:
from ipywidgets import interact
import bokeh.io
import bokeh.models as bkm

def roundToNearest(a, step):
    """Example: round 34.6 to the nearest 5 -> roundToNearest(34.6,5) -> 35"""
    return round(float(a) / step) * step

def convertMatplotlibColorMapToBokeh(mplColormap):
    """example usage: `convertMatplotlibColorMapToBokeh(plt.cm.coolwarm_r)`"""
    import matplotlib.colors
    import bokeh.models as bkm

    colormap = mplColormap
    colorList = [matplotlib.colors.rgb2hex(m) for m in colormap(np.arange(colormap.N))]
    return bkm.mappers.LinearColorMapper(palette=colorList,high=1,low=0)

class trainedModel(fk.LearningModelWithMonitoring):
    def __init__(self,trainDataset,trainLabels,hiddenUnits=[],testDataset=None,testLabels=None,plotStep=0.05,numberOfTrainingSteps=1001):
        # create plot dataset. This bit could be shortened. Also testDataset doesn't do anything just now
        self.plotStep = plotStep
        if testDataset is None:
            self._createPlotCoordinates(trainDataset)
        else:
            self._createPlotCoordinates(np.vstack([trainDataset,testDataset]))
        self._createPlotDataset()
        
        self.trainingDataset = trainDataset
        self.trainingLabels = trainLabels
        self.numberOfTrainingSteps = numberOfTrainingSteps
        self.graphStandIn = fk.DeepClassifierGraph([trainDataset.shape[-1]],trainLabels.shape[-1],hiddenUnits,dropoutKeepProbability=1.0)
        self.batch_size = len(self.trainingDataset) # non stochastic gradient descent
        
        self.stepsToLog = self.calculateStepsToLog(maxNumberOfSteps=15)
        self.trainingRunLog={}
        self.runSession()
        
        self.trainingRunData = pd.DataFrame(self.trainingRunLog).T
        self.closeSession()
        #    if testDataset is not None:
        #        self.testAccuracy = self._calculateAccuracy(self._testPredictionsOp.eval(), testLabels)
        #    self.weights = session.run(self._weightOps)
        #    self.biases = session.run(self._biasOps)
    def logExtraTrainingStats(self,trainingEventData):
        feed_dict = {self.graphStandIn.inputPlaceholder: self.plotDataset}
        trainingEventData['decisionSurface'] = self.session.run(self.graphStandIn.prediction,feed_dict=feed_dict)
        return trainingEventData
    
    def _createPlotCoordinates(self,dataset):
        #set the bounds. These are the bounds of the data extended by 10% and then rounded to the nearest step.
        xmin = roundToNearest(dataset[:,0].min() - 0.1*np.ptp(dataset[:,0]),self.plotStep)
        xmax = roundToNearest(dataset[:,0].max() + 0.1*np.ptp(dataset[:,0]),self.plotStep)
        ymin = roundToNearest(dataset[:,1].min() - 0.1*np.ptp(dataset[:,1]),self.plotStep)
        ymax = roundToNearest(dataset[:,1].max() + 0.1*np.ptp(dataset[:,1]),self.plotStep)
        
        self.xCoords = np.arange(xmin,xmax,self.plotStep)# probablity for max will not be calcuated
        self.yCoords = np.arange(ymin,ymax,self.plotStep)
        
    def _createPlotDataset(self):
        # create a list of xy coordinate values to input to the classifier
        # The loop is constructed specially so that it can be reshaped later for plotting
        xyCoords = []
        for y in self.yCoords: #The last coordinate is for the plotting function and probabilities will not be calculated for it
            for x in self.xCoords:
                xyCoords.append([x,y])
        self.plotDataset = np.asarray(xyCoords,dtype=np.float32)
        
    def plotDecisionSurface(self,plotValues):
        xyValues = plotValues[:,0] #take the first column, don't need both for 2 classes

        #Prepare data for plotting. Each probability will be plotted as a square centered on its xy coord
        halfStep=self.plotStep/2.0
        plotMatrix = xyValues.reshape([self.yCoords.shape[0],self.xCoords.shape[0]])
        xPlotCoords = np.append(self.xCoords,self.xCoords[-1]+self.plotStep)-halfStep
        yPlotCoords = np.append(self.yCoords,self.yCoords[-1]+self.plotStep)-halfStep

        # plot the decision surface
        plt.pcolor(xPlotCoords,yPlotCoords,plotMatrix,cmap=plt.cm.coolwarm,vmin=0,vmax=1)#vmin sets the colorbar range
        plt.xlim([(self.xCoords-halfStep).min(),(self.xCoords-halfStep).max()])
        plt.ylim([(self.yCoords-halfStep).min(),(self.yCoords-halfStep).max()])
        plt.gca().set_aspect('equal')
        plt.colorbar()
        
        # plot the data points
        class1 = self.trainingDataset[(self.trainingLabels == 1)[:,0]]
        class2 = self.trainingDataset[(self.trainingLabels == 0)[:,0]]
        plt.scatter(class1[:,0],class1[:,1],marker='.',color=(1., 0.8508430609597925, 0.2))
        plt.scatter(class2[:,0],class2[:,1],marker='.',color=(0., 0.8982986190585184, 1.))
        
        plt.gcf().set_size_inches([15,6])
        
    def interactivePlot(self):
        loggedSteps = sorted(self.trainingRunLog.keys())
        def plotForStep(stepIndex):
            step = loggedSteps[stepIndex]
            self.plotDecisionSurface(self.trainingRunLog[step]['decisionSurface'])
            print("step: {:<5}   loss: {:5.4f}   trainAccuracy: {:6.3f}%".format(step,self.trainingRunLog[step]['loss'],100*self.trainingRunLog[step]['trainAccuracy']))
        interact(plotForStep,stepIndex=[0,len(loggedSteps)-1])

    def interactivePlotUsingBokeh(self):
        #Prepare all data for plotting
        plotMatrices=[]
        loggedSteps = sorted(self.trainingRunLog.keys())
        for step in loggedSteps:
            plotValues = self.trainingRunLog[step]['decisionSurface']
            xyValues = plotValues[:,0] #take the first column, don't need both for 2 classes

            #Each probability will be plotted as a square centered on its xy coord
            plotMatrix = xyValues.reshape([self.yCoords.shape[0],self.xCoords.shape[0]])
            #xPlotCoords = np.append(self.xCoords,self.xCoords[-1]+self.plotStep)-halfStep
            #yPlotCoords = np.append(self.yCoords,self.yCoords[-1]+self.plotStep)-halfStep
            plotMatrices.append(plotMatrix) #add on an extra dimension because the bokeh plotting function
        #plotMatrices = plotMatrices[0:4]# debugging

        # Get values needed for plotting
        halfStep=self.plotStep/2.0

        xmin=self.xCoords[0]-halfStep
        xmax=self.xCoords[-1]+halfStep
        ymin=self.yCoords[0]-halfStep
        ymax=self.yCoords[-1]+halfStep
        width=xmax-xmin
        height=ymax-ymin

        #Create The Plot
        fig = bkp.figure(
            plot_width=500,
            x_range=(xmin,xmax),
            y_range=(ymin,ymax),
            toolbar_location='right',
            logo=None,
            tools = ['pan','resize','wheel_zoom','reset'],
            min_border_right=10,
            min_border_left=10,
            min_border_top=10,
            min_border_bottom=10)
        fig.plot_height = int(round((fig.plot_width/(width))*(height)))

        # I could set this up with seperate data sources to be neater. See http://nbviewer.jupyter.org/github/bokeh/bokeh-notebooks/blob/master/tutorial/A1%20-%20Building%20gapminder.ipynb
        #make sure everything in here is the same length, otherwise bokeh breaks and needs a restart
        dataSource = bkm.ColumnDataSource({'plotData':[plotMatrices[0]],'plotMatrices':plotMatrices,'stepLabel':[0]})

        colorMapper = convertMatplotlibColorMapToBokeh(plt.cm.coolwarm)

        fig.image(image='plotData',x=[xmin],y=[ymin],dw=width,dh=height,color_mapper=colorMapper,source=dataSource)

        class1 = self.trainingDataset[(self.trainingLabels == 1)[:,0]]
        class2 = self.trainingDataset[(self.trainingLabels == 0)[:,0]]
        fig.scatter(x=class1[:,0],y=class1[:,1],color=(255, 217, 51),size=2)
        fig.scatter(x=class2[:,0],y=class2[:,1],color=(0, 229, 255),size=2)

        fig.text(x=xmin+0.1*width,y=ymax-0.2*height,text='stepLabel',text_font_size='30pts',text_color='yellow',source=dataSource)

        changePlotData = bkm.CustomJS(args={'source':dataSource}, code="""
            var data = source.get('data');
            var sliderValue = cb_obj.get('value');
            var loggedSteps = {}
            data['plotData']=[data['plotMatrices'][sliderValue]];
            data['stepLabel']=[loggedSteps[sliderValue]]
            source.trigger('change');
        """.format(loggedSteps))

        slider = bkm.Slider(start=0, end=len(plotMatrices)-1, value=0, step=1, title="stepIndex",callback=changePlotData)

        interactivePlot = bokeh.io.vform(slider, fig)
        bkp.show(interactivePlot)
        #return interactivePlot

Improvements:
 - Modify the interactive plot function to not flicker when updating the plot. Removing the print statement makes it not so bad
 - fix the weight and bias initialisation for very deep networks
 - make the colorbar's size behave
 - make it work for more than two classes
   - maybe xyProbs = np.abs(xyProbs[:,0]-xyProbs[:,1])

### Color Maps In Bokeh
Bokeh `image()` accepts `color_mapper=` arg which is a `bokeh.models.mappers.LinearColorMapper` object.  
A `LinearColorMapper` is created from a list of color values